In [118]:
import pandas as pd 
import datetime as dt
import numpy as np

In [113]:
gas = pd.read_csv("Data/avg_gas.csv", dtype= {"Date" : str, "Fee (USD)" : float}, parse_dates=[0], infer_datetime_format=True, index_col = ["Date"])
prices = pd.read_csv("Data/hourly_prices.csv", dtype = {"HOUR" : str, "PRICE" : float}, parse_dates = [0], infer_datetime_format=True, index_col = ["HOUR"])
prices = pd.DataFrame(prices["PRICE"])
prices.columns = ["price"]
prices = prices.set_index(pd.to_datetime(prices.index))
tx = pd.read_csv("Data/transactionsJan2022.csv", dtype = str, parse_dates = [32], infer_datetime_format=True, low_memory=False, index_col=[32])
tx_subset = tx[["borrowRate","amountUSD"]].dropna()
tx_subset = tx_subset.astype(float)
tx_subset = tx_subset.set_index(pd.to_datetime(tx_subset.index.strftime('%Y-%m-%d %H:00:00')))
tx_subset["gas"] = gas
tx_subset["price"] = prices
tx_subset["next_price"] = tx_subset["price"].shift()
tx_subset["ret"] = tx_subset["next_price"] / tx_subset["price"] - 1

In [159]:
train_weight = 0.8
split = int(len(tx_subset)*train_weight)
df_train = tx_subset.iloc[:split].copy()
df_test = tx_subset.iloc[split:].copy()

for col in ["borrowRate", "amountUSD","gas"]:
    mu = df_train[col].mean()
    sigma = df_train[col].std()
    print(col, mu, sigma)
    df_train.loc[:,col] = (df_train[col] - mu ) / sigma
    df_test.loc[:,col] = (df_test.loc[:,col] - mu) / sigma

borrowRate -3.791707354490484e-20 1.0
amountUSD 2.7759186765490334e-17 1.0
gas 7.47585790529092e-18 1.0000000000000002
